In [48]:
import os, pathlib

In [49]:
rootDataPath = pathlib.Path('./croped_data/')

In [50]:
babi = [str(path) for path in list(rootDataPath.glob('Babi/*'))]
calimerio = [str(path) for path in list(rootDataPath.glob('Calimerio/*'))]
chrysanthemum = [str(path) for path in list(rootDataPath.glob('Chrysanthemum/*'))]
rosy = [str(path) for path in list(rootDataPath.glob('Rosy/*'))]
tana = [str(path) for path in list(rootDataPath.glob('Tana/*'))]
pingpong = [str(path) for path in list(rootDataPath.glob('Pingpong/*'))]
hydrangeas = [str(path) for path in list(rootDataPath.glob('Hydrangeas/*'))]
lisianthus = [str(path) for path in list(rootDataPath.glob('Lisianthus/*'))]

In [51]:
import pandas as pd

In [52]:
df = pd.DataFrame(columns=["Path", "Type"])

In [53]:
classNames = {'babi': 0, 'calimerio': 1, 'chrysanthemum': 2, 'rosy': 3, 'tana': 4
              , 'pingpong': 5, 'hydrangeas': 6, 'lisianthus': 7}

In [54]:
for path in babi:
    if (".DS_Store" in path):
        continue
    row = pd.DataFrame([{"Path": path, "Type": "0"}])
    df = pd.concat([df, row], axis=0, ignore_index=True)

In [55]:
for path in calimerio:
    if (".DS_Store" in path):
        continue
    row = pd.DataFrame([{"Path": path, "Type": "1"}])
    df = pd.concat([df, row], axis=0, ignore_index=True)

In [56]:
for path in chrysanthemum:
    if (".DS_Store" in path):
        continue
    row = pd.DataFrame([{"Path": path, "Type": "2"}])
    df = pd.concat([df, row], axis=0, ignore_index=True)

In [57]:
for path in rosy:
    if (".DS_Store" in path):
        continue
    row = pd.DataFrame([{"Path": path, "Type": "3"}])
    df = pd.concat([df, row], axis=0, ignore_index=True)

In [58]:
for path in tana:
    if (".DS_Store" in path):
        continue
    row = pd.DataFrame([{"Path": path, "Type": "4"}])
    df = pd.concat([df, row], axis=0, ignore_index=True)

In [59]:
for path in pingpong:
    if (".DS_Store" in path):
        continue
    row = pd.DataFrame([{"Path": path, "Type": "5"}])
    df = pd.concat([df, row], axis=0, ignore_index=True)

In [60]:
for path in hydrangeas:
    if (".DS_Store" in path):
        continue
    row = pd.DataFrame([{"Path": path, "Type": "6"}])
    df = pd.concat([df, row], axis=0, ignore_index=True)

In [61]:
for path in lisianthus:
    if (".DS_Store" in path):
        continue
    row = pd.DataFrame([{"Path": path, "Type": "7"}])
    df = pd.concat([df, row], axis=0, ignore_index=True)

In [62]:
df.tail()

,Path,Type
5162,croped_data/Lisianthus/lisianthus_525.jpg,7
5163,croped_data/Lisianthus/lisianthus_531.jpg,7
5164,croped_data/Lisianthus/lisianthus_257.jpg,7
5165,croped_data/Lisianthus/lisianthus_280.jpg,7
5166,croped_data/Lisianthus/lisianthus_294.jpg,7


<a id='3.2'></a>
### **3.2.** Scale down
***

In [63]:
import cv2

In [64]:
RESIZED_DIMENSION = (32,32)

In [85]:
# Using iterrows() function to loop through the dataframe
for idx, row in df.iterrows():
    print('Scaling image', idx, "|", row['Path'])

    imgName = row['Path'].split('/')[-1] # Get image name
    imgName = imgName.replace('.jpg', '')
    img = cv2.imread(row['Path'])
    resizedImg = cv2.resize(img, RESIZED_DIMENSION)
    cv2.imwrite('./cleansed_data/' + row['Type'] + '/' + imgName + '.jpg', resizedImg)

print('Scale down done!')    

Scaling image 0 | croped_data/Babi/babi_926.jpg
Scaling image 1 | croped_data/Babi/babi_729.jpg
Scaling image 2 | croped_data/Babi/babi_701.jpg
Scaling image 3 | croped_data/Babi/babi_715.jpg
Scaling image 4 | croped_data/Babi/babi_587.7.jpg
Scaling image 5 | croped_data/Babi/babi_522.2.jpg
Scaling image 6 | croped_data/Babi/babi_259.jpg
Scaling image 7 | croped_data/Babi/babi_265.jpg
Scaling image 8 | croped_data/Babi/babi_503.jpg
Scaling image 9 | croped_data/Babi/babi_675.8.jpg
Scaling image 10 | croped_data/Babi/babi_517.jpg
Scaling image 11 | croped_data/Babi/babi_721.5.jpg
Scaling image 12 | croped_data/Babi/babi_271.jpg
Scaling image 13 | croped_data/Babi/babi_488.jpg
Scaling image 14 | croped_data/Babi/babi_12.jpg
Scaling image 15 | croped_data/Babi/babi_339.jpg
Scaling image 16 | croped_data/Babi/babi_762.3.jpg
Scaling image 17 | croped_data/Babi/babi_311.jpg
Scaling image 18 | croped_data/Babi/babi_477.jpg
Scaling image 19 | croped_data/Babi/babi_701.0.jpg
Scaling image 20 | 

<a id='3.3'></a>
### **3.3.** Data augmentation
***

In [80]:
from imblearn.over_sampling import RandomOverSampler
from imblearn.tensorflow import balanced_batch_generator

from keras.utils import Sequence
from keras.preprocessing.image import ImageDataGenerator

In [81]:
class BalancedDataGenerator(Sequence):
    """ImageDataGenerator + RandomOversampling"""
    def __init__(self, input_df, y, datagen, batch_size=32):
        x = np.array([np.array(PIL.Image.open(f).convert('RGB').resize((224, 224))) for f in input_df['ImagePath']])
        # print(len(x))
        y = y.values
        self.datagen = datagen
        self.batch_size = batch_size
        self.gen, self.steps_per_epoch = balanced_batch_generator(x.reshape(x.shape[0], -1), y, sampler=RandomOverSampler(), batch_size=batch_size, keep_sparse=True)
        
    def __len__(self):
        return self.steps_per_epoch

    def __getitem__(self, idx):
        x_batch, y_batch = self.gen.__next__()
        x_batch = x_batch.reshape((-1, 224, 224, 3)) # Reshape to 4D tensor
        x_batch = self.datagen.flow(x_batch, batch_size=self.batch_size, shuffle=False).next()
        return x_batch, y_batch

# Load the images into a NumPy array

# Normalize the image data
datagen = ImageDataGenerator(
                rotation_range=20,
                width_shift_range=0.1,
                height_shift_range=0.1,
                shear_range=0.2,
                zoom_range=0.2,
                horizontal_flip=True,
                vertical_flip=True,
                fill_mode='nearest',
                brightness_range=(0.5,1),
                rescale=1./255)

<a id='3.5'></a>
### **3.5.** Dataset split
***

In [66]:
from sklearn.model_selection import train_test_split

In [67]:
train_data, test_data = train_test_split(df, test_size=0.2, random_state=48, shuffle=True)

In [68]:
train_data, val_data = train_test_split(train_data, test_size=0.25, random_state=48)

In [69]:
print("Train data : {}, Val Data: {}, Test Data: {}".format(train_data.shape[0], val_data.shape[0], test_data.shape[0]))

Train data : 3099, Val Data: 1034, Test Data: 1034


In [70]:
train_data.head()

,Path,Type
2334,croped_data/Rosy/rosy_111.jpg,3
1380,croped_data/Calimerio/calimero_128.jpg,1
2930,croped_data/Tana/tana_442.jpg,4
834,croped_data/Babi/babi_779.9.jpg,0
4582,croped_data/Lisianthus/lisianthus_42.jpg,7


## 1. Classifier

In [1]:
import torchvision
# Initialize pre-trained model
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained = True)

/Users/thuc/opt/anaconda3/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/thuc/opt/anaconda3/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [2]:
model.eval()

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

In [71]:
import tensorflow as tf

In [82]:
INPUT_DIM = (32,32,3)
OUTPUT_CLASSES = 8

In [83]:
HIDDEN_LAYER_DIM = 256

In [ ]:
model = tf.keras.Sequential([
    # Input layer: Conv2D take input as 2D-array suitable for image classification
    # 3D for volumetric data, 4D for video, Transpose for image generation
    #  filters = 64 suitable for image classification (32 or 64)
    #  kernel_size = (3,3) suitable for image classification
    #  padding = "same" output same size as input which also mean strides = 1
    # activation function: - introduce non-linearity into the model important to learn complex patterns in data
    #                      - efficient to train quickly
    #                      - its stable small chance to cause the model to diverge
    tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3), padding="same", activation="relu"),

    # To Downsampling the feature map output from the prev layer. Reduce the size of feature mpas without losing too much info
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),

    # Stack other Convelutional layer to extract higher level features
    tf.keras.layers.Conv2D(
        filters=64,
        kernel_size=(3, 3),
        strides=(1, 1),
        padding="same",
        activation="relu",
    ),

    # To Downsampling the feature map output from the prev layer.
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),

     # Stack other Convelutional layer to extract higher level features
    tf.keras.layers.Conv2D(
        filters=128,
        kernel_size=(3, 3),
        strides=(1, 1),
        padding="same",
        activation="relu",
    ),

    # To Downsampling the feature map output from the prev layer.
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    
])